# Data Base

Florian EPAIN
20105403, MASTER IL group 3

## Belatedly Installation

We can visualize blank `.csv` file using `vscode` and the extension `Rainbow CSV`.

### Docs

- I used the [Psycopg2 tutorials](https://www.psycopg.org/docs/install.html#quick-install) to connect my notebook to my database.
- [SQL Datatypes](https://www.w3schools.com/sql/sql_datatypes.asp)

In [ ]:
!pip install -U pip

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 17.6 MB/s eta 0:00:00m eta 0:00:010:01


In [ ]:
!pip install duckdb==0.9.1

In [1]:
# If the Server postgres is down
import duckdb
cursor = duckdb.connect()
# print(cursor.execute('SELECT 42').fetchall())

In [1]:
import psycopg2
from getpass import getpass
  
# Connection establishment 
conn = psycopg2.connect( 
    database="fepain", 
    user='fepain', 
    password=getpass(), 
    host='chausey.istic.univ-rennes1.fr'
)
  
conn.autocommit = True
  
# Creating a cursor object 
cursor = conn.cursor()

········


In [9]:
# Query to create a table 
sql = ''' CREATE TABLE TripData 
( 
    medallion VARCHAR(32), 
    hack_licence VARCHAR(32), 
    vendor_id VARCHAR(10), 
    rate_code INT,
    store_and_fwd_flag VARCHAR(10),
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count INT,
    trip_time_in_secs INT,
    trip_distance FLOAT,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT
); '''
  
# --- Executing above query ---
cursor.execute(sql) # duckdb & postgres
print("Table has been created successfully D:")

Table has been created successfully D:


---

We now import the `csv` file, using the [Cursor Copy - tutorials](https://www.psycopg.org/docs/cursor.html#cursor.copy_from) and the [COPY docs](https://www.postgresql.org/docs/current/sql-copy.html). I can't use `copy_from()`, I will use `copy_expert()` as mentionned in this [github issue](https://github.com/psycopg/psycopg2/issues/1294)

---

In [6]:
# Display the table 
cursor.execute('SELECT * FROM TripData')
print(cursor.fetchall())

[]


In [2]:
# Import Sample CSV in PostgreSQL
csv_file_path = "assets/data/trip_data-sample.csv"
# cursor.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

# Display the table 
cursor.execute('SELECT * FROM TripData')
print(cursor.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.978165, 40.757977, -73.989838, 40.751171), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.006683, 40.731781, -73.994499, 40.75066), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.1, -74.004707, 40.73777, -74.009834, 40.726002), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.7, -73.974602, 40.759945, -73.984734, 40.759388), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786

In [22]:
from sys import stdout

cursor.copy_expert("COPY TripData TO STDOUT WITH CSV HEADER", stdout)

medallion,hack_licence,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude


In [3]:
# Import Sample CSV in DuckDB

# cursor.execute('SELECT * FROM \'trip_data-sample.csv\';')
# print(cursor.fetchall())

sql = ''' COPY TripData FROM 'trip_data-sample.csv' With CSV HEADER DELIMITER as ','; '''
cursor.execute(sql)
print(cursor.fetchall())

[(9,)]


In [8]:
# Drop the table 
cursor.execute('DROP TABLE TripData')

---

Big CSV Import

---

In [11]:
# Import CSV in PostgreSQL
csv_file_path = "trip_data_1.csv"
# cursor.copy_from(f, 'TripData')

with open(csv_file_path, 'r') as f:
    cursor.copy_expert(
    "COPY TripData FROM STDIN With CSV HEADER DELIMITER as ','",
    f)

In [3]:
# Import Big CSV in DuckDB

# cursor.execute('SELECT * FROM \'trip_data_1.csv\';')
# print(cursor.fetchall())

sql = ''' COPY Trip FROM 'trip_data_1.csv' With CSV HEADER DELIMITER as ','; '''
cursor.execute(sql)
print(cursor.fetchall())

[(14776615,)]


# Analyze Queries


- [EXPLAIN docs](https://www.postgresql.org/docs/current/sql-explain.html)
- <https://explain.dalibo.com/>

In [9]:
# Query 0
query = 'SELECT * FROM TripData WHERE medallion = \'0BD7C8F5BA12B88E0B67BED28BEA73D8\';'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.00668334960938, 40.731781005859375, -73.99449920654297, 40.75065994262695), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.100000023841858, -74.00470733642578, 40.737770080566406, -74.00983428955078, 40.72600173950195)]
-------------
[('physical_plan', '┌───────────────────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│        hack_licence       │\n│         vendor_id         │\n│         rate_code         │\n│     store_and_fwd_flag    │\n│      pickup_datetime      │\n│      dropoff_datetime     │\n│      passenger_count      │\n│     trip_time_

In [4]:
# Query 1
query = 'SELECT * FROM TripData WHERE passenger_count >= 2;'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('89D227B655E5C82AECF13C3F540D4CF4', 'BA96DE419E711691B9445D6A6307C170', 'CMT', 1, 'N', datetime.datetime(2013, 1, 1, 15, 11, 48), datetime.datetime(2013, 1, 1, 15, 18, 10), 4, 382, 1.0, -73.97816467285156, 40.75797653198242, -73.98983764648438, 40.75117111206055), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 54, 15), datetime.datetime(2013, 1, 7, 23, 58, 20), 2, 244, 0.699999988079071, -73.97460174560547, 40.759944915771484, -73.98473358154297, 40.7593879699707), ('0B57B9633A2FECD3D3B1944AFC7471CF', 'CCD4367B417ED6634D986F573A552A62', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 12, 39, 18), datetime.datetime(2013, 1, 7, 13, 10, 56), 3, 1898, 10.699999809265137, -73.98993682861328, 40.75677490234375, -73.86524963378906, 40.7706298828125)]
-------------
[('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│        hack_

In [5]:
# Query 2
query = 'SELECT * FROM TripData WHERE passenger_count = 1;'
cursor.execute(query)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + query
cursor.execute(explain)
print(cursor.fetchall())

[('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 6, 0, 18, 35), datetime.datetime(2013, 1, 6, 0, 22, 54), 1, 259, 1.5, -74.00668334960938, 40.731781005859375, -73.99449920654297, 40.75065994262695), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', '9FD8F69F0804BDB5549F40E9DA1BE472', 'CMT', 1, 'N', datetime.datetime(2013, 1, 5, 18, 49, 41), datetime.datetime(2013, 1, 5, 18, 54, 23), 1, 282, 1.100000023841858, -74.00470733642578, 40.737770080566406, -74.00983428955078, 40.72600173950195), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', '51EE87E3205C985EF8431D850C786310', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 23, 25, 3), datetime.datetime(2013, 1, 7, 23, 34, 24), 1, 560, 2.0999999046325684, -73.97624969482422, 40.74852752685547, -74.0025863647461, 40.747867584228516), ('20D9ECB2CA0767CF7A01564DF2844A3E', '598CCE5B9C1918568DEE71F43CF26CD2', 'CMT', 1, 'N', datetime.datetime(2013, 1, 7, 15, 27, 48), datetime.datetime(2013, 1, 7, 15, 38, 37),

In [6]:
# Aggregate 1
aggregate = 'SELECT medallion, AVG(trip_distance) FROM TripData WHERE passenger_count = 1 GROUP BY medallion;'
cursor.execute(aggregate)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print(cursor.fetchall())

[('2C0E91FF20A856C891483ED63589F982', 0.800000011920929), ('0BD7C8F5BA12B88E0B67BED28BEA73D8', 1.300000011920929), ('DFD2202EE08F7A8DC9A57B02ACB81FE2', 2.0999999046325684), ('496644932DF3932605C22C7926FF0FE0', 0.800000011920929), ('20D9ECB2CA0767CF7A01564DF2844A3E', 1.7000000476837158)]
-------------
[('physical_plan', '┌───────────────────────────┐\n│       HASH_GROUP_BY       │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             #0            │\n│          avg(#1)          │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│   CAST(trip_distance AS   │\n│           DOUBLE)         │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│         medallion         │\n│       trip_distance       │\n

In [7]:
# Aggregate 2
aggregate = 'SELECT AVG(passenger_count) FROM TripData GROUP BY pickup_datetime;'
cursor.execute(aggregate)
print(cursor.fetchall())

print("-------------")

explain = 'EXPLAIN ' + aggregate
cursor.execute(explain)
print(cursor.fetchall())

[(1.0,), (2.0,), (3.0,), (4.0,), (1.0,), (1.0,), (1.0,), (1.0,), (1.0,)]
-------------
[('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│    avg(passenger_count)   │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│       HASH_GROUP_BY       │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│             #0            │\n│          avg(#1)          │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         PROJECTION        │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      pickup_datetime      │\n│      passenger_count      │\n└─────────────┬─────────────┘                             \n┌─────────────┴─────────────┐\n│         SEQ_SCAN          │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│       TripDataSample      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│      pickup_datetime      │\n│      passenger_count      │\n│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │\n│           EC: 9           │\n

In [8]:
import time

# Aggregate 3
start_time = time.perf_counter()
number_of_iteration = 1000;

for x in range(number_of_iteration):

    query = 'SELECT medallion, AVG(passenger_count) as avg_pass_count, AVG(trip_distance) as avg_trip_dist FROM TripData GROUP BY medallion'
    cursor.execute(query)
    # print('(medallion, avg_passenger_count, avg_trip_dist):')
    # print(cursor.fetchall())

    # print("-------------")

    global_avg = 'SELECT AVG(trip_distance) FROM TripData'
    cursor.execute(global_avg)
    # print('global avg trip distance:')
    # print(cursor.fetchall())

    # print("-------------")

    aggregate = 'SELECT avg_pass_count FROM (' + query + ') WHERE avg_trip_dist > (' + global_avg + ');'
    cursor.execute(aggregate)
    # print('A3:')
    # print(cursor.fetchall())

    # print("-------------")

    explain = 'EXPLAIN ' + aggregate
    cursor.execute(explain)
    # print(cursor.fetchall())

end_time = time.perf_counter()
time_lapsed = end_time - start_time
print('Average Time Elapsed / sql execution (in sec):')
print(time_lapsed / number_of_iteration)

Average Time Elapsed / sql execution (in sec):
0.0017032031920007284


## Compare DuckDB and PostgreSQL

- OLAP vs OLTP

In [13]:
# Closing the connection 
conn.close()